# Introduction to Functional Programming in Haskell

## Equational Reasoning

 * Definitions in Haskell are actually that: definitions.
 * The rule is, if you have a definition **`LHS = RHS`**, then anywhere you see **`LHS`** in your code you can replace it with **`RHS`** and get the same result.
 * This means we cannot have multiple assignments of a variable in the same scope.

In [ ]:
x = 1
x

In [ ]:
x = 3
x = 4
x

## Types

 * Every expression in Haskell has a **type**.
 * We read the code "**`expr :: Type`**" as "expression **`expr`** has type **`Type`**".

In [ ]:
x = 3 :: Int

In [ ]:
:t x

In [ ]:
:t (x, x)

In [ ]:
:t [x, x, x, x]

In [ ]:
:t fst (x, x)

## Type Constraints

In [ ]:
:t 3

In [ ]:
:info Num

## Function Types (Currying)

In [ ]:
:t (+)

In [ ]:
:t (+ x)

In [ ]:
:t x + x

## Using Higher-Order Functions

In [ ]:
:t map

In [ ]:
:t map (+ x)

In [ ]:
map (+ 2) [1 .. 5]

In [ ]:
:t filter

In [ ]:
filter even [1..5]

In [ ]:
:t (.)

In [ ]:
:t filter (> 0) . map (+ x)

In [ ]:
:t foldl1

In [ ]:
foldl1 (+) [1..10]

## Non-Strict Evaluation

In a typical language, what happens when you evaluate something like this?

**```
x = (f(), g());
return x[0];```**

In [ ]:
:t error

In [ ]:
:t error "Fail!"

In [ ]:
error "Fail!"

In [ ]:
(1, error "Fail!")

In [ ]:
fst (1, error "Fail!")

## Some Fun Stuff: Infinite Sequences

In [ ]:
list = map (+ 3) [1..] :: [Int]

In [ ]:
take 4 list

In [ ]:
fib = 1 : 1 : zipWith (+) fib (tail fib)

In [ ]:
take 10 fib

In [ ]:
take 5 (drop 50 fib)

## The Mathy Stuff: Functors

In [ ]:
:info Functor